In [1]:
library(lme4)
library(simr)
library(dplyr)
library(broom)
install.packages("sjPlot")
library(sjPlot)
install.packages("broom.mixed")
library(broom.mixed)
library(htmlTable)
library(dplyr)
library(stringr)
# library(gt)
library(car)
install.packages("stargazer")
library(stargazer)
# First install and load lmerTest
install.packages("lmerTest")
library(lmerTest)

Loading required package: Matrix


Attaching package: ‘simr’


The following object is masked from ‘package:lme4’:

    getData



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union





The downloaded binary packages are in
	/var/folders/h_/pm5z54tn7rj170br8494ljz80000gn/T//RtmpJDSr7Y/downloaded_packages

The downloaded binary packages are in
	/var/folders/h_/pm5z54tn7rj170br8494ljz80000gn/T//RtmpJDSr7Y/downloaded_packages



Attaching package: ‘stringr’


The following object is masked from ‘package:simr’:

    fixed


Loading required package: carData


Attaching package: ‘car’


The following object is masked from ‘package:dplyr’:

    recode





The downloaded binary packages are in
	/var/folders/h_/pm5z54tn7rj170br8494ljz80000gn/T//RtmpJDSr7Y/downloaded_packages



Please cite as: 


 Hlavac, Marek (2022). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R package version 5.2.3. https://CRAN.R-project.org/package=stargazer 





The downloaded binary packages are in
	/var/folders/h_/pm5z54tn7rj170br8494ljz80000gn/T//RtmpJDSr7Y/downloaded_packages



Attaching package: ‘lmerTest’


The following object is masked from ‘package:lme4’:

    lmer


The following object is masked from ‘package:stats’:

    step




In [2]:

file_path_total <- 'pilot_total_stats/distractor_wm_ltm.csv'
file_path_5 <- 'pilot5_stats/distractor_wm_ltm.csv'
file_path_6 <- 'pilot6_stats/distractor_wm_ltm.csv'

df_total <- read.csv(file_path_total) 
df5 <- read.csv(file_path_5) 
df6 <- read.csv(file_path_6) 


In [3]:
control <- lmerControl(optimizer = "bobyqa")#, optCtrl = list(maxfun = 10000))

In [4]:
run_ltm_models <- function(df) {
    
   
    
    model_abs <- glmer(recall_answ ~ (it_sim_dis_test_z + it_sim_dis_test_sq_z)  * reliability_binary_z *  
                      validity_binary_z + (v2_sim_dis_test_z + v2_sim_dis_test_sq_z)  * reliability_binary_z *  
                      validity_binary_z + (1 | participant), 
                      data = df, family = binomial,control = glmerControl(optimizer = "bobyqa",
                                            optCtrl = list(maxfun = 500000),
                                            calc.derivs = FALSE))
    
    
    model_rel <- glmer(recall_answ ~ (it_sim_dis_diff_test_z + it_sim_dis_diff_test_sq_z) * reliability_binary_z *  
                      validity_binary_z + (v2_sim_dis_diff_test_z + v2_sim_dis_diff_test_sq_z)  * reliability_binary_z *  
                      validity_binary_z + (1 | participant), 
                      data = df, family = binomial,control = glmerControl(optimizer = "bobyqa",
                                            optCtrl = list(maxfun = 500000),
                                            calc.derivs = FALSE))

    model_rel_split <- glmer(recall_answ ~ (it_sim_dis_diff_test_sign_z + it_sim_dis_diff_test_sign_sq_z) * it_pos_neg * reliability_binary_z *  
                      validity_binary_z + (v2_sim_dis_diff_test_sign_z + v2_sim_dis_diff_test_sign_sq_z) * v2_pos_neg * reliability_binary_z *  
                      validity_binary_z + (1 | participant), 
                      data = df, family = binomial,control = glmerControl(optimizer = "bobyqa",
                                            optCtrl = list(maxfun = 500000),
                                            calc.derivs = FALSE))
return(list( 
    model_abs       = model_abs,
    model_rel       = model_rel,
    model_rel_split = model_rel_split
  ))
}



In [5]:
results_old_stims <- run_ltm_models(df6)
results_new_stims <- run_ltm_models(df5)
results_total <- run_ltm_models(df_total)




In [6]:
extract_significance <- function(model_summary) {
  # For glmer models
  if ("coefficients" %in% names(model_summary)) {
    # Check more carefully for z value vs t value columns
    coef_table <- model_summary$coefficients
    var_names <- rownames(coef_table)
    
    # Check which p-value column exists
    if ("Pr(>|z|)" %in% colnames(coef_table)) {
      p_values <- coef_table[, "Pr(>|z|)"]
    } else if ("Pr(>|t|)" %in% colnames(coef_table)) {
      p_values <- coef_table[, "Pr(>|t|)"]
    } else {
      # If there's a z or t value, calculate p-values manually
      if ("z value" %in% colnames(coef_table)) {
        z_values <- coef_table[, "z value"]
        p_values <- 2 * pnorm(abs(z_values), lower.tail = FALSE)
      } else if ("t value" %in% colnames(coef_table)) {
        t_values <- coef_table[, "t value"]
        # Using a large df for approximation
        p_values <- 2 * pt(abs(t_values), df = 100, lower.tail = FALSE)
      } else {
        stop("Cannot find p-values or z/t values in coefficients table")
      }
    }
  } else {
    # More detailed debug info
    cat("Available names in model summary:\n")
    print(names(model_summary))
    
    # Try to access fixed effects via other means
    if ("fixed.effects" %in% names(model_summary)) {
      var_names <- names(model_summary$fixed.effects)
      # Manually calculate p-values using fixed effects and std errors
      # This is just a placeholder - would need to be adapted to actual structure
      stop("Found fixed.effects but need appropriate calculation of p-values")
    } else {
      stop("Could not extract coefficients from model summary")
    }
  }
  
  # Calculate significance stars
  significance <- sapply(p_values, function(p) {
    if (is.na(p)) return("")
    if (p < 0.001) return("***")
    if (p < 0.01) return("**")
    if (p < 0.05) return("*")
    if (p < 0.1) return(".")
    return("")
  })
  
  return(data.frame(
    variable = var_names,
    p_value = p_values,
    significance = significance,
    stringsAsFactors = FALSE
  ))
}


In [7]:
compare_models <- function(model_list, model_names, 
                          include_p_values = FALSE,
                          only_significant = TRUE,
                          significance_level = 0.1) {
  
  sig_list <- lapply(model_list, extract_significance)
  names(sig_list) <- model_names
  
  all_vars <- unique(unlist(lapply(sig_list, function(x) x$variable)))
  
  result_df <- data.frame(Variable = all_vars, stringsAsFactors = FALSE)
  
  # First, determine which rows have significance in any model
  has_significance_by_var <- sapply(all_vars, function(var) {
    any_significant <- FALSE
    for(model_name in model_names) {
      model_sig <- sig_list[[model_name]]
      idx <- which(model_sig$variable == var)
      if(length(idx) > 0) {
        stars <- model_sig$significance[idx]
        if(stars != "") {
          any_significant <- TRUE
          break
        }
      }
    }
    return(any_significant)
  })
  
  # Now populate the data frame with all p-values for rows with any significance
  for(model_name in model_names) {
    model_sig <- sig_list[[model_name]]
    result_df[[model_name]] <- sapply(all_vars, function(var) {
      idx <- which(model_sig$variable == var)
      if(length(idx) == 0) return("")
      
      p_val <- model_sig$p_value[idx]
      stars <- model_sig$significance[idx]
      
      # Always show p-value if this variable has significance in any model
      if(has_significance_by_var[var] || !only_significant) {
        return(paste0(stars, " (p=", sprintf("%.4f", p_val), ")"))
      } else if(stars != "") {
        # Show significance stars with p-value for significant results
        return(paste0(stars, " (p=", sprintf("%.4f", p_val), ")"))
      } else {
        # For non-significant results in rows without any significance
        return("")
      }
    })
  }
  
  # Filter rows if only_significant is TRUE
  if(only_significant) {
    result_df <- result_df[has_significance_by_var, ]
  }
  
  # Adjust HTML table styling to make first column wider
  html_table <- htmlTable(
    result_df,
    caption = "Significance Comparison Across Models",
    rnames = FALSE,
    css.cell = "padding: 8px;",
    css.table = "border-collapse: collapse; width: 100%; font-family: Arial, sans-serif;",
    css.header = "background-color: #f2f2f2; font-weight: bold; border: 1px solid #dddddd;",
    css.rgroup = "border-bottom: 1px solid #dddddd;",
    css.tspanner = "font-weight: bold; text-align: center; border-bottom: 2px solid #dddddd;",
    css.class = "table-hover"
  )
  
  # Add additional CSS to make the first column wider
  html_table <- gsub("<table", "<style>td:first-child, th:first-child { min-width: 300px; max-width: 500px; word-wrap: break-word; }</style><table", html_table)
  
  return(html_table)
}

In [8]:

model_abs_old <- summary(results_old_stims$model_abs)
model_abs_new <- summary(results_new_stims$model_abs)
model_abs_total <- summary(results_total$model_abs)

comparison_table_abs <- compare_models(
  model_list = list(model_abs_old, model_abs_new, model_abs_total),
  model_names = c("Old Stimuli", "New Stimuli", "Total"),
  include_p_values = TRUE,  # Change to FALSE for just stars
  only_significant = TRUE
)

# Save to HTML file for inclusion in presentations
# writeLines(comparison_table_abs, "comparison_table_distractor_abs_cent.html")



model_rel_old <- summary(results_old_stims$model_rel)
model_rel_new <- summary(results_new_stims$model_rel)
model_rel_total <- summary(results_total$model_rel)

comparison_table_rel <- compare_models(
  model_list = list(model_rel_old, model_rel_new, model_rel_total),
  model_names = c("Old Stimuli", "New Stimuli", "Total"),
  include_p_values = TRUE,  # Change to FALSE for just stars
  only_significant = TRUE
)

# Save to HTML file for inclusion in presentations
# writeLines(comparison_table_rel, "comparison_table_distractor_rel_cent.html")



model_rel_split_old <- summary(results_old_stims$model_rel_split)
model_rel_split_new <- summary(results_new_stims$model_rel_split)
model_rel_split_total <- summary(results_total$model_rel_split)

comparison_table_rel_split <- compare_models(
  model_list = list(model_rel_split_old, model_rel_split_new, model_rel_split_total),
  model_names = c("Old Stimuli", "New Stimuli", "Total"),
  include_p_values = TRUE,  # Change to FALSE for just stars
  only_significant = TRUE
)

# Save to HTML file for inclusion in presentations
# writeLines(comparison_table_rel_split, "comparison_table_distractor_rel_split_cent.html")
writeLines(comparison_table_abs, "comparison_table_distractor_abs_rt.html")
writeLines(comparison_table_rel, "comparison_table_distractor_rel_rt.html")
writeLines(comparison_table_rel_split, "comparison_table_distractor_rel_split_rt.html")


In [9]:
# IRdisplay::display_html(comparison_table_abs_split)
IRdisplay::display_html(comparison_table_abs)


Significance Comparison Across Models 
 
 Variable 
 Old Stimuli 
 New Stimuli 
 Total 
 
 
 
 
 (Intercept) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 
 
 it_sim_dis_test_z 
 * (p=0.0398) 
 ** (p=0.0044) 
 *** (p=0.0006) 
 
 
 it_sim_dis_test_sq_z 
 ** (p=0.0045) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 
 
 reliability_binary_z:v2_sim_dis_test_z 
 (p=0.8380) 
 * (p=0.0489) 
 (p=0.5853)

In [10]:
IRdisplay::display_html(comparison_table_rel)
IRdisplay::display_html(comparison_table_rel_split)


Significance Comparison Across Models 
 
 Variable 
 Old Stimuli 
 New Stimuli 
 Total 
 
 
 
 
 (Intercept) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 
 
 it_sim_dis_diff_test_z 
 (p=0.3702) 
 ** (p=0.0060) 
 * (p=0.0100) 
 
 
 it_sim_dis_diff_test_sq_z 
 *** (p=0.0009) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 
 
 v2_sim_dis_diff_test_sq_z 
 *** (p=0.0003) 
 (p=0.3210) 
 *** (p=0.0007) 
 
 
 it_sim_dis_diff_test_sq_z:validity_binary_z 
 (p=0.8282) 
 . (p=0.0837) 
 (p=0.1450) 
 
 
 it_sim_dis_diff_test_z:reliability_binary_z:validity_binary_z 
 (p=0.9141) 
 * (p=0.0349) 
 . (p=0.0934)

Significance Comparison Across Models 
 
 Variable 
 Old Stimuli 
 New Stimuli 
 Total 
 
 
 
 
 (Intercept) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 
 
 it_sim_dis_diff_test_sign_z 
 (p=0.4026) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 
 
 it_sim_dis_diff_test_sign_sq_z 
 *** (p=0.0000) 
 * (p=0.0421) 
 ** (p=0.0011) 
 
 
 v2_sim_dis_diff_test_sign_z 
 ** (p=0.0017) 
 (p=0.5178) 
 *** (p=0.0008) 
 
 
 v2_sim_dis_diff_test_sign_sq_z 
 (p=0.3799) 
 * (p=0.0422) 
 (p=0.5331) 
 
 
 it_sim_dis_diff_test_sign_z:it_pos_neg 
 (p=0.4037) 
 * (p=0.0112) 
 ** (p=0.0094) 
 
 
 it_sim_dis_diff_test_sign_z:validity_binary_z 
 (p=0.5064) 
 . (p=0.0988) 
 . (p=0.0583) 
 
 
 reliability_binary_z:validity_binary_z 
 (p=0.9421) 
 . (p=0.0759) 
 (p=0.2695) 
 
 
 it_sim_dis_diff_test_sign_sq_z:it_pos_neg:reliability_binary_z 
 (p=0.2355) 
 (p=0.2666) 
 . (p=0.0793) 
 
 
 it_sim_dis_diff_test_sign_z:it_pos_neg:validity_binary_z 
 (p=0.8770) 
 . (p=0.0600) 
 (p=0.2252) 
 
 
 it_sim_dis_diff_test_sign_sq_z:it_pos_neg:validity_binary_z 
 (p=0.2203) 
 (p=0.4017) 
 * (p=0.0335) 
 
 
 it_pos_neg:reliability_binary_z:validity_binary_z 
 (p=0.7541) 
 * (p=0.0223) 
 (p=0.1295) 
 
 
 reliability_binary_z:v2_sim_dis_diff_test_sign_sq_z:v2_pos_neg 
 (p=0.4274) 
 * (p=0.0148) 
 (p=0.3860) 
 
 
 validity_binary_z:v2_sim_dis_diff_test_sign_z:v2_pos_neg 
 (p=0.8190) 
 . (p=0.0553) 
 (p=0.4155) 
 
 
 validity_binary_z:v2_sim_dis_diff_test_sign_sq_z:v2_pos_neg 
 (p=0.1299) 
 . (p=0.0896) 
 * (p=0.0241) 
 
 
 it_sim_dis_diff_test_sign_sq_z:it_pos_neg:reliability_binary_z:validity_binary_z 
 (p=0.8781) 
 . (p=0.0692) 
 (p=0.2382) 
 
 
 reliability_binary_z:validity_binary_z:v2_sim_dis_diff_test_sign_z:v2_pos_neg 
 (p=0.3341) 
 . (p=0.0699) 
 (p=0.5456)

In [11]:
# Change the modeling function from glmer (for binary data) to lmer (for continuous data)
run_rt_models <- function(df) {
    # First, let's rename the dataset column to make it clearer
    df$rt_ltm <- df$image_recall_response.rt
    df_correct <- subset(df, recall_answ == 1)
    df_correct$log_rt_ltm <- log(df_correct$rt_ltm)

    cols_to_scale <- c("reliability_binary_z", "validity_binary_z", 
                  "it_sim_dis_test_z", "it_sim_dis_test_sq_z", "v2_sim_dis_test_z", "v2_sim_dis_test_sq_z",
                   "it_sim_dis_diff_test_z", "it_sim_dis_diff_test_sq_z","v2_sim_dis_diff_test_z", "v2_sim_dis_diff_test_sq_z", 
                      "it_sim_dis_diff_test_sign_z", "it_sim_dis_diff_test_sign_sq_z","v2_sim_dis_diff_test_sign_z", "v2_sim_dis_diff_test_sign_sq_z")

    df_correct[paste0(cols_to_scale, "_new")] <- lapply(df_correct[cols_to_scale], scale)
    
    # Now create the models using lmer instead of glmer
    model_abs <- lmer(log_rt_ltm ~ (it_sim_dis_test_z_new + it_sim_dis_test_sq_z_new) * reliability_binary_z_new *  
                      validity_binary_z_new + (v2_sim_dis_test_z_new + v2_sim_dis_test_sq_z_new) * reliability_binary_z_new *  
                      validity_binary_z_new + (1 | participant), 
                      data = df_correct, control = lmerControl(optimizer = "bobyqa"))
    
    model_rel <- lmer(log_rt_ltm ~ (it_sim_dis_diff_test_z_new + it_sim_dis_diff_test_sq_z_new) * reliability_binary_z_new *  
                      validity_binary_z_new + (v2_sim_dis_diff_test_z_new + v2_sim_dis_diff_test_sq_z_new) * reliability_binary_z_new *  
                      validity_binary_z_new + (1 | participant), 
                      data = df_correct, control = lmerControl(optimizer = "bobyqa"))

    model_rel_split <- lmer(log_rt_ltm ~ (it_sim_dis_diff_test_sign_z_new + it_sim_dis_diff_test_sign_sq_z_new) * it_pos_neg * reliability_binary_z_new *  
                      validity_binary_z_new + (v2_sim_dis_diff_test_sign_z_new + v2_sim_dis_diff_test_sign_sq_z_new) * v2_pos_neg * reliability_binary_z_new *  
                      validity_binary_z_new + (1 | participant), 
                      data = df_correct, control = lmerControl(optimizer = "bobyqa"))

    
   


    
    return(list( 
      model_abs       = model_abs,
      model_rel       = model_rel,
      model_rel_split = model_rel_split
    ))
}

In [12]:
results_rt_old_stims <- run_rt_models(df6)
results_rt_new_stims <- run_rt_models(df5)
results_rt_total <- run_rt_models(df_total)

In [13]:
model_rt_abs_old <- summary(results_rt_old_stims$model_abs)
model_rt_abs_new <- summary(results_rt_new_stims$model_abs)
model_rt_abs_total <- summary(results_rt_total$model_abs)

comparison_table_rt_abs <- compare_models(
  model_list = list(model_rt_abs_old, model_rt_abs_new, model_rt_abs_total),
  model_names = c("Old Stimuli", "New Stimuli", "Total"),
  include_p_values = TRUE,
  only_significant = TRUE
)

# Save the RT comparison tables
writeLines(comparison_table_rt_abs, "comparison_table_distractor_rt_abs.html")

# Relative similarity models
model_rt_rel_old <- summary(results_rt_old_stims$model_rel)
model_rt_rel_new <- summary(results_rt_new_stims$model_rel)
model_rt_rel_total <- summary(results_rt_total$model_rel)

comparison_table_rt_rel <- compare_models(
  model_list = list(model_rt_rel_old, model_rt_rel_new, model_rt_rel_total),
  model_names = c("Old Stimuli", "New Stimuli", "Total"),
  include_p_values = TRUE,
  only_significant = TRUE
)

writeLines(comparison_table_rt_rel, "comparison_table_distractor_rt_rel.html")

# Split relative similarity models
model_rt_rel_split_old <- summary(results_rt_old_stims$model_rel_split)
model_rt_rel_split_new <- summary(results_rt_new_stims$model_rel_split)
model_rt_rel_split_total <- summary(results_rt_total$model_rel_split)

comparison_table_rt_rel_split <- compare_models(
  model_list = list(model_rt_rel_split_old, model_rt_rel_split_new, model_rt_rel_split_total),
  model_names = c("Old Stimuli", "New Stimuli", "Total"),
  include_p_values = TRUE,
  only_significant = TRUE
)

writeLines(comparison_table_rt_rel_split, "comparison_table_distractor_rt_rel_split.html")

# Display the RT comparison tables

In [14]:
IRdisplay::display_html(comparison_table_rt_abs)
IRdisplay::display_html(comparison_table_rt_rel)
IRdisplay::display_html(comparison_table_rt_rel_split)

Significance Comparison Across Models 
 
 Variable 
 Old Stimuli 
 New Stimuli 
 Total 
 
 
 
 
 (Intercept) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 
 
 v2_sim_dis_test_z_new 
 (p=0.4998) 
 (p=0.1485) 
 * (p=0.0124) 
 
 
 v2_sim_dis_test_sq_z_new 
 (p=0.9025) 
 . (p=0.0946) 
 ** (p=0.0098) 
 
 
 reliability_binary_z_new:validity_binary_z_new 
 * (p=0.0442) 
 (p=0.2242) 
 * (p=0.0146) 
 
 
 it_sim_dis_test_z_new:reliability_binary_z_new:validity_binary_z_new 
 (p=0.9946) 
 ** (p=0.0042) 
 * (p=0.0322) 
 
 
 it_sim_dis_test_sq_z_new:reliability_binary_z_new:validity_binary_z_new 
 (p=0.5544) 
 * (p=0.0345) 
 (p=0.2218) 
 
 
 reliability_binary_z_new:validity_binary_z_new:v2_sim_dis_test_sq_z_new 
 * (p=0.0242) 
 (p=0.5652) 
 (p=0.2850)

Significance Comparison Across Models 
 
 Variable 
 Old Stimuli 
 New Stimuli 
 Total 
 
 
 
 
 (Intercept) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 
 
 it_sim_dis_diff_test_sq_z_new 
 (p=0.1268) 
 * (p=0.0416) 
 * (p=0.0229) 
 
 
 reliability_binary_z_new:validity_binary_z_new 
 . (p=0.0639) 
 (p=0.1987) 
 * (p=0.0279) 
 
 
 reliability_binary_z_new:validity_binary_z_new:v2_sim_dis_diff_test_sq_z_new 
 (p=0.3396) 
 . (p=0.0845) 
 (p=0.7307)

Significance Comparison Across Models 
 
 Variable 
 Old Stimuli 
 New Stimuli 
 Total 
 
 
 
 
 (Intercept) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 
 
 it_sim_dis_diff_test_sign_z_new 
 (p=0.1024) 
 * (p=0.0426) 
 * (p=0.0185) 
 
 
 v2_pos_neg 
 (p=0.1410) 
 . (p=0.0969) 
 (p=0.6501) 
 
 
 it_sim_dis_diff_test_sign_z_new:it_pos_neg 
 * (p=0.0430) 
 (p=0.4567) 
 (p=0.8001) 
 
 
 it_sim_dis_diff_test_sign_sq_z_new:reliability_binary_z_new 
 . (p=0.0510) 
 (p=0.5847) 
 * (p=0.0478) 
 
 
 reliability_binary_z_new:validity_binary_z_new 
 * (p=0.0371) 
 (p=0.1936) 
 * (p=0.0146) 
 
 
 validity_binary_z_new:v2_sim_dis_diff_test_sign_sq_z_new 
 * (p=0.0432) 
 (p=0.4090) 
 (p=0.3543) 
 
 
 it_sim_dis_diff_test_sign_sq_z_new:it_pos_neg:validity_binary_z_new 
 * (p=0.0365) 
 (p=0.8907) 
 (p=0.1266) 
 
 
 reliability_binary_z_new:validity_binary_z_new:v2_sim_dis_diff_test_sign_sq_z_new 
 (p=0.4281) 
 . (p=0.0689) 
 (p=0.6052) 
 
 
 it_sim_dis_diff_test_sign_z_new:it_pos_neg:reliability_binary_z_new:validity_binary_z_new 
 (p=0.3776) 
 * (p=0.0311) 
 . (p=0.0546) 
 
 
 it_sim_dis_diff_test_sign_sq_z_new:it_pos_neg:reliability_binary_z_new:validity_binary_z_new 
 (p=0.1752) 
 (p=0.1014) 
 * (p=0.0464)